In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
path = '/content/drive/MyDrive'
os.chdir(path)

!source venv_d2l/bin/activate

In [ ]:
## !pip install d2l

!pip install git+https://github.com/d2l-ai/d2l-en.git

  Cloning https://github.com/d2l-ai/d2l-en.git to /tmp/pip-req-build-ibsfnqgx
  Running command git clone --filter=blob:none --quiet https://github.com/d2l-ai/d2l-en.git /tmp/pip-req-build-ibsfnqgx
  Resolved https://github.com/d2l-ai/d2l-en.git to commit 23d7a5aecceee57d1292c56e90cce307f183bb0a
  Preparing metadata (setup.py) ... done


In [ ]:
path = '/content/drive/MyDrive/d2l-zh'
os.chdir(path)

In [ ]:
# 多GPU训练 从零开始
%matplotlib inline
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l

In [ ]:
# 简单网络
# 把lenet那一节的代码抄了过来
# lenet是非常小的网络 用他来做样本网络来测试多GPU训练
# 这个选择也会带来问题 --
scale = 0.01
W1 = torch.randn(size=(20, 1, 3, 3)) * scale
b1 = torch.zeros(20)
W2 = torch.randn(size=(50, 20, 5, 5)) * scale
b2 = torch.zeros(50)
W3 = torch.randn(size=(800, 128)) * scale
b3 = torch.zeros(128)
W4 = torch.randn(size=(128, 10)) * scale
b4 = torch.zeros(10)
params = [W1, b1, W2, b2, W3, b3, W4, b4]

def lenet(X, params):
    h1_conv = F.conv2d(input=X, weight=params[0], bias=params[1])
    h1_activation = F.relu(h1_conv)
    h1 = F.avg_pool2d(input=h1_activation, kernel_size=(2, 2), stride=(2, 2))
    h2_conv = F.conv2d(input=h1, weight=params[2], bias=params[3])
    h2_activation = F.relu(h2_conv)
    h2 = F.avg_pool2d(input=h2_activation, kernel_size=(2, 2), stride=(2, 2))
    h2 = h2.reshape(h2.shape[0], -1)
    h3_linear = torch.mm(h2, params[4]) + params[5]
    h3 = F.relu(h3_linear)
    y_hat = torch.mm(h3, params[6]) + params[7]
    return y_hat

loss = nn.CrossEntropyLoss(reduction='none')

In [ ]:
# 向多个设备分发参数
# 如果p已经在GPU上了，保险起见，用clone才会再复制
def get_params(params, device):
    new_params = [p.clone().to(device) for p in params]
    for p in new_params:
        p.requires_grad_()
    return new_params

# 开始lenet的参数全部都在CPU上的，现在我们把他放在我们的GPU0上
new_params = get_params(params, d2l.try_gpu(0))
print('b1 weight:', new_params[1])
print('b1 gradient:', new_params[1].grad)

In [ ]:
# allreduce函数将所有向量相加，并将结果广播给所有GPU
# data是个list，里面有4个GPU的话，list有4个元素
# 把gpu0以外的gpu都先复制到gpu0上 to(data[0].device)，然后再相加
# 这样data就是所有data的求和，就把这个新的结果复制回去
def allreduce(data):
    for i in range(1, len(data)):
        data[0][:] += data[i].to(data[0].device)
    for i in range(1, len(data)):
        data[i] = data[0].to(data[i].device)

data = [torch.ones((1, 2), device=d2l.try_gpu(i)) * (i + 1) for i in range(2)]
print('before allreduce：\n', data[0], '\n', data[1])
allreduce(data)
print('after allreduce：\n', data[0], '\n', data[1])

In [ ]:
# 将一个小批量数据均匀地分布再多个GPU上
data = torch.arange(20).reshape(4, 5)
devices = [torch.device('cuda:0'), torch.device('cuda:1')]
# 这里使用了nn.parallel.scatter函数，data就是一个tensor，他就均匀地切开到gpu上
# gpu0上前两行，gpu1上后两行
split = nn.parallel.scatter(data, devices)
print('input :', data)
print('load into', devices)
print('output:', split)

In [ ]:
def split_batch(X, y, devices):
    """将X和y拆分到多个设备上
    devices是GPU列表
    """
    assert X.shape[0] == y.shape[0]
    return (nn.parallel.scatter(X, devices),
            nn.parallel.scatter(y, devices))
# 把样本和y都均匀地分配到GPU上

In [ ]:
# 在一个小批量上实现多GPU训练
# 输入小批量X,y, device_params是所有GPU上的参数，devices是我要知道用哪些GPU，lr是学习率
# 用split_batch函数把小批量均匀地分配到GPU上
def train_batch(X, y, device_params, devices, lr):
    X_shards, y_shards = split_batch(X, y, devices)
    # 对每个GPU上分别计算自己那个小批量上的样本损失
    # w是完整的，X_shard, y_shard是里面的一块，把x和w放入lenet里，对y求loss再求和
    # 返回的ls是给list，对应每个GPU上对应的损失
    ls = [loss(lenet(
        X_shard, device_W), y_shard).sum()
          for X_shard, y_shard, device_W in zip(
              X_shards, y_shards, device_params)]
    for l in ls: # 反向传播在每个GPU上分别执行
        l.backward()
    # 将每个GPU的所有梯度相加，并将其广播到所有GPU
    # i是你的层，每一层对于所有的GPU，把梯度拿出来，对他做allreduce
    # 这样每层的grad是替换操作，是整个小批量完整的梯度
    with torch.no_grad():
        for i in range(len(device_params[0])):
            allreduce(
                [device_params[c][i].grad for c in range(len(devices))])
    # 在每个GPU上分别更新模型参数
    for param in device_params:
        d2l.sgd(param, lr, X.shape[0]) # 在这里，我们使用全尺寸的小批量
# 这里有个问题，发现有一点串行，这就看框架在背后能不能帮你做并行了
# 框架做的好的话算ls和backward是可以自动并行的 mxnet和tensorflow是没问题的，pytorch有点悬


In [ ]:
# 定义训练函数
def train(nums_gpus, batch_size, lr):
    # 创建数据iterator
    train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)
    # 不同的就是告诉要用多少个gpu
    devices = [d2l.try_gpu(i) for i in range(nums_gpus)]
    # 将模型参数复制到num_gpus个GPU
    # 训练开始前把初始化的params复制到每个gpu上
    device_params = [get_params(params, d) for d in devices]
    num_epochs = 10
    # 后面没什么区别，画个东西，记个时
    animator = d2l.Animator('epoch', 'test acc', xlim=[1, num_epochs])
    timer = d2l.Timer()
    # 对于每个epoch，每个iterator抽个X,y
    for epoch in range(num_epochs):
        timer.start()
        for X, y in train_iter:
            # 用train_batch做一次小批量更新
            train_batch(X, y, device_params, devices, lr)
            # cuda.synchronize保证每个gpu都做完了这件事，主要为了算时间准一点
            torch.cuda.synchronize()
        timer.stop()
        # 在GPU0上评估模型
        animator.add(epoch + 1, (d2l.evaluate_accuracy_gpu(
            lambda x: lenet(x, device_params[0]), test_iter, devices[0]),))
    # 打印结果
    print(f'test acc：{animator.Y[0][-1]:.2f}，{timer.avg():.1f}sec/epoch，'
          f'on {str(devices)}')

In [ ]:
# 在单个GPU上运行
train(num_gpus=1, batch_size=256, lr=0.2)

In [ ]:
# 增加为2个GPU
train(num_gpus=2, batch_size=256, lr=0.2)

In [ ]:
# 两个的结果都是2.6 sec/epoch, 多GPU不变快或者分布式不变快这是非常常见的现象
# 通常的原因有
# 一是data读取就很慢
# 二是gpu增加了但是batch size没增加，之前一个gpu能处理256个样本每一次，这时候2个gpu，每个gpu只能处理128个样本，相当于batch size变低了计算性能就变低，gpu有几千个线程，
# 因为batch数量变小，做同样操作的效率会变低，无法多线程，没有打满gpu。遇到这种情况在增加gpu的时候，要保证每个gpu还是拿到同样多的batch size
# 直接*2的话会发现精度可能会变低，因为lr不变，收敛很可能会变慢，性能稍微好了一点，从2.6到2.4sec/epoch
# 解决测试精度的做法是把learning rate调大一点，如果batch size特别大的时候，lr也不能无限增加，测试精度也不会有原来的效果
# 三是这里理论上每个gpu的batch size都和原来一样，理论上性能应该比较好了，但是还是只有2.4，这里也有两个原因
# 1. pytorch对并行的加速不够好，裸写的写法对pytorch是不友好的
# 2. lenet太小了，性能确实没那么好
train(num_gpus=2, batch_size=256*2, lr=0.2*1.5)

In [ ]:
# 多GPU的简洁实现
import torch
from torch import nn
from d2l import torch as d2l

In [ ]:

def resnet18(num_classes, in_channels=1):
    """稍加修改的ResNet-18模型"""
    def resnet_block(in_channels, out_channels, num_residuals,
                     first_block=False):
        blk = []
        for i in range(num_residuals):
            if i == 0 and not first_block:
                blk.append(d2l.Residual(in_channels, out_channels,
                                        use_1x1conv=True, strides=2))
            else:
                blk.append(d2l.Residual(out_channels, out_channels))
        return nn.Sequential(*blk)

    # 该模型使用了更小的卷积核、步长和填充，而且删除了最大汇聚层
    net = nn.Sequential(
        nn.Conv2d(in_channels, 64, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(64),
        nn.ReLU())
    net.add_module("resnet_block1", resnet_block(
        64, 64, 2, first_block=True))
    net.add_module("resnet_block2", resnet_block(64, 128, 2))
    net.add_module("resnet_block3", resnet_block(128, 256, 2))
    net.add_module("resnet_block4", resnet_block(256, 512, 2))
    net.add_module("global_avg_pool", nn.AdaptiveAvgPool2d((1,1)))
    net.add_module("fc", nn.Sequential(nn.Flatten(),
                                       nn.Linear(512, num_classes)))
    return net

In [ ]:
net = resnet18(10)
# 获取GPU列表
devices = d2l.try_all_gpus()
# 我们将在训练代码实现中初始化网络

TypeError: Residual.__init__() got multiple values for argument 'use_1x1conv'

In [ ]:
def train(net, num_gpus, batch_size, lr):
    train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)
    devices = [d2l.try_gpu(i) for i in range(num_gpus)]
    def init_weights(m):
        if type(m) in [nn.Linear, nn.Conv2d]:
            nn.init.xavier_uniform_(m.weight, std=0.01)
    net.apply(init_weights)
    # 在多个GPU上设置模型
    net = nn.DataParallel(net, device_ids=devices)
    trainer = torch.optim.SGD(net.parameters(), lr)
    loss = nn.CrossEntropyLoss()
    timer, num_epochs = d2l.Timer(), 10
    animator = d2l.Animator('epoch', 'test acc', xlim=[1, num_epochs])
    for epoch in range(num_epochs):
        net.train()
        timer.start()
        for X, y in train_iter:
            trainer.zero_grad()
            X, y = X.to(devices[0]), y.to(devices[0])
            l = loss(net(X), y)
            l.backward()
            trainer.step()
        timer.stop()
        animator.add(epoch + 1, (d2l.evaluate_accuracy_gpu(net, test_iter),))
    print(f'测试精度：{animator.Y[0][-1]:.2f}，{timer.avg():.1f}秒/轮，'
          f'在{str(devices)}')

In [ ]:
train(net, num_gpus=1, batch_size=256, lr=0.1)

In [ ]:
train(net, num_gpus=2, batch_size=512, lr=0.2)